# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [2]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [3]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -6.65278625 -1.5        -0.          0.
  6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [ ]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [100]:
# Network parameters
HIDDEN_IN = 256
HIDDEN_OUT = 128

# Agent parameters
BUFFER_SIZE = int(1e6)
BATCH_SIZE = 128
GAMMA = 0.99
TAU = 7e-2
LR_ACTOR = 1e-3
LR_CRITIC = 1e-3
WEIGHT_DECAY = 0
NOISE_DECAY = 0.999

# Training parameters
N_EPISODES = 50
PRINT_EVERY = 10

# Seed
RANDOM_SEED = 10

# Noise params
MU = 0.
THETA = 0.12
SIGMA = 0.2
SCALE = 1
EPS_INIT = 3     
EPS_END = 200     
EPS_LAST = 0     

NUM_AGENTS = 2
REPEAT_LEARNING = 2
UPDATE_EVERY = 1

In [101]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


# normalize hidden layers when init
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1.0 / np.sqrt(fan_in)
    return (-lim, lim)


# Actor Network
class Actor(nn.Module):
    # Actor class deciding actions in the environment
    def __init__(self, state_size, action_size, seed=RANDOM_SEED, hidden_in=HIDDEN_IN, hidden_out=HIDDEN_OUT):

        """ PARAMETERS
            ==========
                state_size (int) = dimension of input space
                action_size (int) = dimension of action space
                seed (int) = level of randomness
                hidden_in/out (int) = number of neurons per hidden layer
            
            RETURNS
            =======
                mapping of input space to action probability

        """

        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        
        # init layers
        self.fc1 = nn.Linear(state_size, hidden_in)
        self.fc2 = nn.Linear(hidden_in, hidden_out)
        self.fc3 = nn.Linear(hidden_out, action_size)

        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)
            

    def forward(self, state):
        
        #if state.dim() == 1:
         #   state = torch.unsqueeze(state, 0)
            
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return x


# Critic TD3 Network
class Critic(nn.Module):
    # Critic TD3 class for Q-value estimation
    def __init__(self, state_size, action_size, seed=RANDOM_SEED, hidden_in=HIDDEN_IN, hidden_out=HIDDEN_OUT):

        """ PARAMETERS
            ==========
                state_size (int) = dimension of input space
                action_size (int) = dimension of action space
                seed (int) = level of randomness
                hidden_in/out (int) = number of neurons per hidden layer
            
            RETURNS
            =======
                mapping of state + action -> Q-values

        """

        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)

        # define Q1 network architecture
        self.fcs1 = nn.Linear(state_size, hidden_in)
        self.fc2 = nn.Linear(hidden_in+action_size, hidden_out)
        self.fc3 = nn.Linear(hidden_out, 1)
        
        # define Q2 network architecture
        self.fcs4 = nn.Linear(state_size, hidden_in)
        self.fc5 = nn.Linear(hidden_in+action_size, hidden_out)
        self.fc6 = nn.Linear(hidden_out, 1)

        self.reset_parameters()


    def reset_parameters(self):
        
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

        self.fcs4.weight.data.uniform_(*hidden_init(self.fcs4))
        self.fc5.weight.data.uniform_(*hidden_init(self.fc5))
        self.fc6.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """ Critic Network maps states and actions to Q-value estimations """
        
        #if state.dim() == 1:
         #   state = torch.unsqueeze(state, 0)
        
        # Q1 forward
        xs1 = F.relu(self.fcs1(state))
        x1 = torch.cat((xs1, action), dim=1)
        x1 = F.relu(self.fc2(x1))
        x1 = self.fc3(x1)
        
        # Q2 forward
        xs2 = F.relu(self.fcs4(state))
        x2 = torch.cat((xs2, action), dim=1)
        x2 = F.relu(self.fc5(x2))
        x2 = self.fc6(x2)
        
        # return both network outputs
        return x1, x2

    def Q1(self, state, action):
        """ Return only output of one sub-network Q1 """
        xs1 = F.relu(self.fcs1(state))
        x1 = torch.cat((xs1, action), dim=1)
        x1 = F.relu(self.fc2(x1))
        x1 = self.fc3(x1)
        return x1

In [102]:
# imports
import torch
import numpy as np
import random
from collections import namedtuple, deque

# create memory
class ReplayBuffer(object):
    
    # Fixed-size buffer to store experience tuples for 
    # sequence-learning in the Tennis environment 

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """ Initialize Replay Buffer.
        ==========
        PARAMETERS
        ==========
            action_size (int) = dimensions of available actions
            buffer_size (int) = maximum length of memory
            batch_size (int) = number of samples to learn from at each iteration
            seed (int) = random seed
        """

        self.seed = random.seed(seed)
        self.batch_size = batch_size
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.experience = namedtuple("Experience", field_names = ["state", "action", 
                                                "reward", "next_state", "done"])

    def add(self, states, actions, rewards, next_states, dones):
        # append experience sequence to memory
        e = self.experience(states, actions, rewards, next_states, dones)
        self.memory.append(e)

    def sample(self):
        # randomly collect batch_size of experiences for learning
        experiences = random.sample(self.memory, k=self.batch_size)

        states = [torch.from_numpy(np.vstack([e.state[idx] for e in experiences if e is not None])).float().to(device) for idx in range(num_agents)]
        actions = [torch.from_numpy(np.vstack([e.action[idx] for e in experiences if e is not None])).float().to(device) for idx in range(num_agents)]
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = [torch.from_numpy(np.vstack([e.next_state[idx] for e in experiences if e is not None])).float().to(device) for idx in range(num_agents)]
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device) 

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        # current memory length
        return len(self.memory)

In [103]:
# imports
import random
import copy
import sys
import numpy as np
import torch


# create action noise
class OUNoise(object):
    # Ornstein-Uhlenbeck noise process

    def __init__(self, action_dimension, seed=RANDOM_SEED, scale=SCALE, mu=MU, theta=THETA, sigma=SIGMA):
        """ Initialize object """
        
        self.size = action_dimension
        self.scale = scale
        self.theta = theta
        self.sigma = sigma
        self.mu = mu * np.ones(action_dimension)
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        # reset noise parameters after each episode
        self.state = copy.copy(self.mu)

    def sample(self):
        # create noise to add to actions
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.standard_normal(self.size)
        self.state = x + dx
        return self.state

In [104]:
class TD3Agent(object):
    """ Twin Delayed Deep Deterministic Policy Gradient Model """

    def __init__(self, state_size, action_size, random_seed, num_agents=NUM_AGENTS):
        """ Initialize the model with arguments as follows:
                
                    ARGUMENTS
                    =========
                        - state_size (int) = dimension of input space
                        - action_size (int) = dimension of action space
                        - random_seed (int) = random seed
                    Returns 
                    =======
                        - best learned action to take after Actor-Critic Learning
        """
        
        self.state_size = state_size
        self.action_size = action_size
        self.num_agents = num_agents
        self.seed = random.seed(random_seed)

        # Actor Networks - private (only see own observation state)
        self.actor_local = Actor(state_size, action_size, random_seed).to(device)
        self.actor_target = Actor(state_size, action_size, random_seed).to(device)
        self.actor_optimizer = torch.optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)

        # Critic Networks
        self.critic_local = Critic(state_size*NUM_AGENTS, action_size*NUM_AGENTS, random_seed).to(device)
        self.critic_target = Critic(state_size*NUM_AGENTS, action_size*NUM_AGENTS, random_seed).to(device)
        self.critic_optimizer = torch.optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)

        # add action noise
        self.noise = OUNoise(action_size)
        #self.noise_decay = NOISE_DECAY

        self.learn_counter = 0
        self.eps = EPS_INIT
        self.eps_decay = 1/(EPS_END*REPEAT_LEARNING)

        # hard copy local weights to target weights
        self.soft_update(self.actor_local, self.actor_target, 1)
        self.soft_update(self.critic_local, self.critic_target, 1)

    def soft_update(self, local_model, target_model, tau):
        # if tau==1 then hard copy is performed
        for local_param, target_param in zip(local_model.parameters(), target_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
            
    def act(self, state, add_noise = True):
        # decide actions from local actor
        state = torch.from_numpy(state).float().to(device)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            action += self.noise.sample() * self.eps
        return np.clip(action, -1, 1)
  
    def reset(self):
        self.noise.reset()
        

In [105]:
# imports
import numpy as np
import random
import copy
import torch
from collections import namedtuple, deque

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class MATD3(object):
    
    def __init__(self, state_size, action_size, n_agents, random_seed):
        super(MATD3, self).__init__()
        
        self.state_size = state_size
        self.action_size = action_size
        self.n_agents = n_agents
        self.seed = random.seed(random_seed)
        
        # init 2 agents
        self.matd3_agents = [TD3Agent(state_size, action_size, random_seed, n_agents),
                             TD3Agent(state_size, action_size, random_seed, n_agents)]
        # shared memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)
           

    def reset(self):
        for agent in self.matd3_agents:
            agent.reset()
            
    def act(self, states):
        actions = np.zeros([num_agents, action_size])
        for idx, agent in enumerate(self.matd3_agents):
            actions[idx,  :] = agent.act(states[idx], add_noise=True)
        return actions
    
    def step(self, states, actions, rewards, next_states, dones, i_episode):
                
        self.memory.add(states, actions, rewards, next_states, dones)
        if (len(self.memory) > BATCH_SIZE) and (i_episode % UPDATE_EVERY == 0):
            for i in range(REPEAT_LEARNING):
                # learn agent1
                experiences = self.memory.sample()
                self.matd3_learn(experiences, self_id=0, non_self_id=1)
                # learn agent2
                experiences = self.memory.sample()
                self.matd3_learn(experiences, self_id=1, non_self_id=0)
                
                
    def matd3_learn(self, experiences, self_id, non_self_id, gamma=GAMMA):
        
        states, actions, rewards, next_states, dones = experiences
        
        self_states = states[self_id]
        self_actions = actions[self_id]
        self_next_states = next_states[self_id]
        
        nself_states = states[non_self_id]
        nself_actions = actions[non_self_id]
        nself_next_states = next_states[non_self_id]

        
        full_states = torch.cat((self_states, nself_states), dim=1).to(device)
        full_actions = torch.cat((self_actions, nself_actions), dim=1).to(device)
        full_next_states = torch.cat((self_next_states, nself_next_states), dim=1).to(device)
        
        agent = self.matd3_agents[self_id]
        
        full_next_actions = torch.cat((self.matd3_agents[self_id].actor_target(self_next_states),
                                       self.matd3_agents[non_self_id].actor_target(nself_next_states)), dim=1).to(device)

        target_Q1, target_Q2 = agent.critic_target(full_next_states, full_next_actions)
        target_Q = torch.min(target_Q1, target_Q2)
                
        target_Q = rewards[self_id] + (gamma * target_Q * (1-dones[self_id]))

        current_Q1, current_Q2 = agent.critic_local(full_states, full_actions)

        # compute critic loss
        critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
        
        # update the critic
        agent.critic_optimizer.zero_grad()
        critic_loss.backward()
        #torch.nn.utils.clip_grad_norm_(agent.critic_local.parameters(), 1)
        agent.critic_optimizer.step()
        
        
        #if self.learn_counter % policy_freq == 0:
        full_actions_pred = torch.cat((self.matd3_agents[self_id].actor_local(self_states),
                                           self.matd3_agents[non_self_id].actor_local(nself_states)), dim=1).to(device)
        actor_loss = -agent.critic_local.Q1(full_states, full_actions_pred).mean()
            
        agent.actor_optimizer.zero_grad()
        actor_loss.backward()
        #torch.nn.utils.clip_grad_norm_(agent.actor_local.parameters(), 1)

        agent.actor_optimizer.step()

        # delay update of actor and critic target models
        agent.soft_update(agent.actor_local, agent.actor_target, TAU)
        agent.soft_update(agent.critic_local, agent.critic_target, TAU)
            
        agent.eps -= agent.eps_decay
        agent.eps = max(agent.eps, EPS_LAST)
        agent.noise.reset()

In [108]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# imports
import time
import numpy as np
import torch
import sys
import os
from collections import deque


def train():
    
    """ Function to run training for multiple agents
        ARGUMENTS
        =========
            - multi_td3 (object) = multiple agents
            - env (object) = Unity ML environment
            - n_episodes (int) = Number of training iterations
            - max_t (int) = maximum timesteps per iteration
            - print_every (int) = how often to output results
            
        RETURNS
        =======
            - scores (int) = scores made per episode
    """
    
    torch.manual_seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    
    
    multi_td3 = MATD3(state_size, action_size, num_agents, RANDOM_SEED)
        
    
    scores_list = []
    scores_deque = deque(maxlen=100)
    avg_scores_list = []

    

    time_global_0 = time.time()
    for i_episode in range(1, 300+1):

        time_episode_0 = time.time()
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        scores = np.zeros(num_agents)
        multi_td3.reset()

        while True: 
            actions = multi_td3.act(states)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            
            multi_td3.step(states, actions, rewards, next_states, dones, i_episode)
            states = next_states
            scores += rewards

            if np.any(dones):
                break

        max_score = np.max(scores)
        scores_list.append(max_score)
        scores_deque.append(max_score)
        avg_score = np.mean(scores_deque)
        avg_scores_list.append(avg_score)
        
        

        print('\rEpisode {} ---> Scores: {}; Average Score: {:.2f}.\t' \
              .format(i_episode, scores, np.mean(scores_deque)), end="")
        
        if i_episode % PRINT_EVERY == 0:
            
            itertime = time.time() - time_episode_0
            elapsedtime = time.time() - time_global_0
            itertime_string = time.strftime('%Hh%Mm%Ss', time.gmtime(itertime))
            
            print('\rEpisode {}, {} ---> Scores: {}/Max:{}; Average Score: {:.2f}.\t' \
              .format(i_episode, itertime_string, scores, np.around(np.max(scores[-PRINT_EVERY:]),decimals=3), np.mean(scores_deque)))
            
        

        if avg_score >= 0.5:
            globaltime_string = time.strftime('%Hh%Mm%Ss', time.gmtime(elapsedtime))
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, avg_score))
            print("==============================================")
            print("Environment solved in {} episodes.\t" \
                  "Average score is: {:.2f}\tElapsed total time: {}".format(i_episode-PRINT_EVERY,
                                                                            avg_score,
                                                                            globaltime_string))
            
            print()
            print("\nSaving final model ...", end="")
            for i in range(num_agents):
                torch.save(multi_td3.matd3_agent[i].actor_local.state_dict(), 'checkpoint_actor_{}.pth'.format(i))
                torch.save(multi_td3.matd3_agent[i].critic_local.state_dict(), 'checkpoint_critic_{}.pth'.format(i))
            print("\nModel saved.")
            break

    return scores_list, scores_deque, avg_scores_list

In [109]:
scores, deque, avg = train()

Episode 10, 00h00m01s ---> Scores: [-0.01  0.  ]/Max:0.0; Average Score: 0.00.	
Episode 20, 00h00m01s ---> Scores: [-0.01  0.  ]/Max:0.0; Average Score: 0.00.	
Episode 30, 00h00m01s ---> Scores: [ 0.   -0.01]/Max:0.0; Average Score: 0.00.	
Episode 40, 00h00m01s ---> Scores: [-0.01  0.  ]/Max:0.0; Average Score: 0.00.	
Episode 50, 00h00m01s ---> Scores: [ 0.   -0.01]/Max:0.0; Average Score: 0.00.	
Episode 60, 00h00m01s ---> Scores: [ 0.   -0.01]/Max:0.0; Average Score: 0.00.	
Episode 70, 00h00m01s ---> Scores: [ 0.   -0.01]/Max:0.0; Average Score: 0.00.	
Episode 80, 00h00m01s ---> Scores: [-0.01  0.  ]/Max:0.0; Average Score: 0.00.	
Episode 90, 00h00m01s ---> Scores: [-0.01  0.  ]/Max:0.0; Average Score: 0.00.	
Episode 100, 00h00m01s ---> Scores: [-0.01  0.  ]/Max:0.0; Average Score: 0.00.	
Episode 110, 00h00m01s ---> Scores: [-0.01  0.  ]/Max:0.0; Average Score: 0.00.	
Episode 120, 00h00m01s ---> Scores: [ 0.   -0.01]/Max:0.0; Average Score: 0.00.	
Episode 130, 00h00m01s ---> Scores: [